# CUHK-STAT3009: Homework 2 - More SVD Models **(due Nov 13)**


## **Q1: Basic Usage of SVD for Rating Prediction**

**Importing the SVD Class**

Download the `SVD` class from our GitHub repository: https://github.com/statmlben/CUHK-STAT3009/blob/main/src/TabRS.py.

**Dataset**

We will use a synthetic dataset to demonstrate the basic usage of SVD for rating prediction. The dataset consists of user ratings for various items, represented by the following DataFrame:
```python
import pandas as pd
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'item_id': [1, 2, 1, 3, 1, 3, 2, 3, 2, 3],
    'rating': [5, 3, 4, 2, 1, 3, 4, 5, 2, 3]}
df = pd.DataFrame(data)
```

**Task**

Your task is to train an SVD model with $K = 2$ and $\lambda = 0.001$ using the provided dataset and predict the ratings for the following user-item pairs:

* `user_id` = 2, `item_id` = 2
* `user_id` = 5, `item_id` = 1

Implement the SVD model, train it on the dataset, and provide the predicted ratings for the specified user-item pairs.

> The correctness of the implementation will be evaluated based on the code structure and logic, not on the final evaluation results.

### Review of Matrix Factorization method in RS

#### Motivation: associate each user and item with latent factors
- latent factors can be seen as a dense vector describing the characteristics of users and items
  - $\mathbf{p}_u = (p_{u1}, \cdots, p_{uK})^T$
  - $\mathbf{q}_i = (q_{i1}, \cdots, q_{iK})^T$
- Rating (user-item interaction) is given by the inner product: $r_{ui} = \mathbf{p}_u^T\mathbf{q}_i$.
  - Why inner product? consider $\cos(\theta_{ui}) = \frac{\mathbf{p}_u^T\mathbf{q}_i}{\|\mathbf{p}_u\| \|\mathbf{q}_i\|}$, where $\theta_{ui}$ is the angle between $\mathbf{p}_u$ and $\mathbf{q}_i$.
  - Suppose the lengths are normalized to 1, $\cos(\theta_{ui})=\mathbf{p}_u^T\mathbf{q}_i$.
  - larger cosine -> smaller angle -> more similar -> higher rating
- Dimension of latent factor $K$ is a hyper-parameter; large $K$ -> describe user/item in a complex way -> (1) model is more powerful if data is large enough; (2) model is overfitting otherwise.

#### Training
- Objective (empirical risk minimization): $$\min \frac{1}{|\Omega|} \sum_{(u,i)\in\Omega} (r_{ui}-\mathbf{p}_u^T\mathbf{q}_i)^2$$

- Regularized Objective (to prevent overfitting) $$\min \frac{1}{|\Omega|} \sum_{(u,i)\in\Omega} (r_{ui}-\mathbf{p}_u^T\mathbf{q}_i)^2 + \lambda (\sum_{u} \|\mathbf{p}_u\|_2^2 + \sum_i \|\mathbf{q}_q\|_2^2)$$

- Optimization
    - objective is nonconvex due to the bilinear term $\mathbf{p}_u^T\mathbf{q}_i$
    - if one of $\mathbf{P}$ or $\mathbf{Q}$ is fixed, then it is convex in $\mathbf{Q}$ or $\mathbf{P}$; moreover, it is essentially a Ridge regression problem
    - Algorithm: Blockwise Coordinate Descent (Alternative Least Squares)
        - alternate between two blocks
        - update all $\mathbf{p}$ s while fixing all $\mathbf{q}$ s
        - update all $\mathbf{q}$ s while fixing all $\mathbf{p}$ s

        \begin{align}
        &\min_{\mathbf{P}, \mathbf{Q}} \frac{1}{|\Omega|} \sum_{(u,i)\in\Omega} (r_{ui}-\mathbf{p}_u^T\mathbf{q}_i)^2 + \lambda (\sum_{u} \|\mathbf{p}_u\|_2^2 + \sum_i \|\mathbf{q}_q\|_2^2) \\
        \Leftrightarrow \quad &\min_{\mathbf{Q}} \frac{1}{|\Omega|} \sum_{(u,i)\in\Omega} (r_{ui}-\mathbf{p}_u^T\mathbf{q}_i)^2 + \lambda \sum_i \|\mathbf{q}_q\|_2^2 \\
        \Leftrightarrow \quad &\min_{\mathbf{Q}} \sum_{i=1}^m (\frac{1}{|\Omega|} \sum_{u} (r_{ui} - \mathbf{p}_u^T\mathbf{q}_i)^2 + \lambda \|\mathbf{q}_i\|_2^2) \tag{Ridge regression for each $i$}
        \end{align}
    - Incorporate bias term

        \begin{align}
            \min_{\mathbf{P}, \mathbf{Q}} \frac{1}{|\Omega|} \sum_{(u,i)\in\Omega} (r_{ui}-\mu - a_u - b_i - \mathbf{p}_u^T\mathbf{q}_i)^2 + \lambda (\sum_{u} \|\mathbf{p}_u\|_2^2 + \sum_i \|\mathbf{q}_q\|_2^2)
        \end{align}
        - Example: fix all others, solve $\mu$; take derivative and set it to zero
        \begin{align}
            \mu = \frac{1}{|\Omega|} \sum_{(u,i)} r_{ui} - a_u - b_i
        \end{align}

In [ ]:
## Your solution here.
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.linear_model import Ridge

## Copy SVD class from GibHub repo
class SVD(BaseEstimator):
    """
    Matrix Factorization (MF) class for collaborative filtering.

    Parameters
    ----------
    n_users : int
        The number of users in the system.
    n_items : int
        The number of items in the system.
    lam : float, optional (default=0.001)
        The regularization parameter.
    K : int, optional (default=10)
        The number of latent factors.
    iterNum : int, optional (default=10)
        The number of iterations for the ALS algorithm.
    tol : float, optional (default=1e-4)
        The tolerance for convergence.
    verbose : int, optional (default=1)
        A flag to control the verbosity of the algorithm.

    Attributes
    ----------
    mu: float
        The global effect of the ratings.
    a : array, shape (n_users)
        The user bias term.
    b : array, shape (n_items)
        The item bias term.
    P : array, shape (n_users, K)
        The user latent factors.
    Q : array, shape (n_items, K)
        The item latent factors.

    Methods
    -------
    fit(X, y)
        Fits the matrix factorization model to the given data.
    predict(X)
        Predicts the ratings for a given set of user-item pairs.
    mse(X, y)
        Computes the mean squared error (MSE) between the predicted ratings and the actual ratings.
    obj(X, y)
        Computes the objective function value, which is the sum of the MSE and the regularization term.

    Notes
    -----
    This implementation uses the Alternating Least Squares (ALS) algorithm to learn the latent factors.
    """

    def __init__(self, n_users, n_items, lam=.01, K=5, iterNum=10, tol=1e-4, verbose=1):
        self.mu = 0.0
        self.a = np.zeros(n_users)
        self.b = np.zeros(n_items)
        self.P = np.random.randn(n_users, K)
        self.Q = np.random.randn(n_items, K)
        self.n_users = n_users
        self.n_items = n_items
        self.K = K
        self.lam = lam
        self.iterNum = iterNum
        self.tol = tol
        self.verbose = verbose

    def fit(self, X, y):
        """
        Fits the matrix factorization model to the given data.

        Parameters
        ----------
        X : array, shape (n_samples, 2)
            A 2D array of shape `(n_samples, 2)`, where each row represents a user-item pair.
        y : array, shape (n_samples,)
            A 1D array of shape `(n_samples,)`, where each element represents the actual rating.

        Returns
        -------
        self
            The fitted model.
        """
        diff = 1.0
        n_users, n_items, n_obs = self.n_users, self.n_items, len(X)
        K, lam = self.K, self.lam

        if self.verbose:
            print('Fitting Reg-SVD: K: %d, lam: %.5f' %(self.K, self.lam))

        self.index_item = [np.where(X[:,1] == i)[0] for i in range(n_items)]
        self.index_user = [np.where(X[:,0] == u)[0] for u in range(n_users)]

        for l in range(self.iterNum):
            obj_old = self.obj(X, y)

            ## update global bias term
            self.mu = np.mean(y - self.predict(X) + self.mu)

            ## update item params
            for item_id in range(n_items):
                index_item_tmp = self.index_item[item_id]
                if len(index_item_tmp) == 0:
                    self.Q[item_id,:] = 0.0
                    continue
                ## update item bias term
                y_tmp = y[index_item_tmp]
                X_tmp = X[index_item_tmp]
                U_tmp = X_tmp[:,0]
                self.b[item_id] = np.mean(y_tmp - self.predict(X_tmp) + self.b[item_id])

                ## update item latent factors
                res_tmp = y_tmp - self.mu - self.b[item_id] - self.a[U_tmp]
                P_tmp = self.P[U_tmp]
                clf = Ridge(alpha=lam*n_obs, fit_intercept=False)
                clf.fit(X=P_tmp, y=res_tmp)
                self.Q[item_id,:] = clf.coef_

            ## update user params
            for user_id in range(n_users):
                index_user_tmp = self.index_user[user_id]
                if len(index_user_tmp) == 0:
                    self.P[user_id,:] = 0.0
                    continue
                ## update item bias term
                y_tmp = y[index_user_tmp]
                X_tmp = X[index_user_tmp]
                I_tmp = X_tmp[:,1]
                self.a[user_id] = np.mean(y_tmp - self.predict(X_tmp) + self.a[user_id])

                ## update user latent factors
                res_tmp = y_tmp - self.mu - self.b[I_tmp] - self.a[user_id]
                Q_tmp = self.Q[I_tmp]
                clf = Ridge(alpha=lam*n_obs, fit_intercept=False)
                clf.fit(X=Q_tmp, y=res_tmp)
                self.P[user_id,:] = clf.coef_

            obj_new = self.obj(X, y)
            diff = abs(obj_old - obj_new)

            rmse_tmp = np.sqrt(self.mse(X, y))
            if self.verbose:
                print("RegSVD-ALS: %d; obj: %.3f; rmse:%.3f, diff: %.3f"
                      %(l, obj_new, rmse_tmp, diff))

            if diff < self.tol:
                break

        return self

    def predict(self, X):
        """
        Predicts the ratings for a given set of user-item pairs.

        Parameters
        ----------
        X : array, shape (n_samples, 2)
            A 2D array of shape `(n_samples, 2)`, where each row represents a user-item pair.

        Returns
        -------
        pred_y : array, shape (n_samples,)
            A 1D array of shape `(n_samples,)`, where each element represents the predicted rating.
        """
        pred_y = [self.mu + self.a[X_tmp[0]] + self.b[X_tmp[1]] + np.dot(self.P[X_tmp[0]], self.Q[X_tmp[1]]) for X_tmp in X]
        # pred_y = self.mu + self.a[X[:,0]] + self.b[X[:,1]] + self.P[X[:,0]].dot(self.Q[X[:,1]].T)
        return np.array(pred_y)

    def mse(self, X, y):
        """
        Computes the mean squared error (MSE) between the predicted ratings and the actual ratings.

        Parameters
        ----------
        X : array, shape (n_samples, 2)
            A 2D array of shape `(n_samples, 2)`, where each row represents a user-item pair.
        y : array, shape (n_samples,)
            A 1D array of shape `(n_samples,)`, where each element represents the actual rating.

        Returns
        -------
        mse : float
            The mean squared error.
        """
        pred_y = self.predict(X)
        return np.mean( (pred_y - y)**2 )

    def obj(self, X, y):
        """
        Computes the objective function value, which is the sum of the MSE and the regularization term.

        Parameters
        ----------
        X : array, shape (n_samples, 2)
            A 2D array of shape `(n_samples, 2)`, where each row represents a user-item pair.
        y : array, shape (n_samples,)
            A 1D array of shape `(n_samples,)`, where each element represents the actual rating.

        Returns
        -------
        obj : float
            The objective function value.
        """
        mse_tmp = self.mse(X, y)
        pen_tmp = np.sum( (self.P)**2 ) + np.sum( (self.Q)**2 )
        return mse_tmp + self.lam*pen_tmp

In [ ]:
import pandas as pd

# generate synthetic dataset
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'item_id': [1, 2, 1, 3, 1, 3, 2, 3, 2, 3],
    'rating': [5, 3, 4, 2, 1, 3, 4, 5, 2, 3]}
df = pd.DataFrame(data)

X_train, y_train = df.values[:, :2], df.values[:, 2]

In [ ]:
# train and predict

K = 2
lam = 0.001

n_users = df['user_id'].max()
n_items = df['item_id'].max()
X_train -= 1  # SVD starts with index 0

svd_rs = SVD(n_users=n_users, n_items=n_items, K=K, lam=lam)
svd_rs.fit(X_train, y_train)

X_test = np.array([
    [2, 2],
    [5, 1]
]) - 1

pred = svd_rs.predict(X_test)

print(f"Prediction: {pred}")

Fitting Reg-SVD: K: 2, lam: 0.00100
RegSVD-ALS: 0; obj: 0.082; rmse:0.021, diff: 14.315
RegSVD-ALS: 1; obj: 0.069; rmse:0.020, diff: 0.012
RegSVD-ALS: 2; obj: 0.060; rmse:0.019, diff: 0.009
RegSVD-ALS: 3; obj: 0.053; rmse:0.018, diff: 0.007
RegSVD-ALS: 4; obj: 0.047; rmse:0.017, diff: 0.006
RegSVD-ALS: 5; obj: 0.042; rmse:0.015, diff: 0.005
RegSVD-ALS: 6; obj: 0.038; rmse:0.014, diff: 0.004
RegSVD-ALS: 7; obj: 0.035; rmse:0.014, diff: 0.003
RegSVD-ALS: 8; obj: 0.032; rmse:0.013, diff: 0.003
RegSVD-ALS: 9; obj: 0.030; rmse:0.013, diff: 0.002
Prediction: [1.01915514 0.34799983]


## **Q2: Lasso-SVD Recommender Systems**

**Data**

In this task, you will implement a user-item average based recommender system using the Netflix dataset from the CUHK-STAT3009 GitHub repository.

```python
import numpy as np
import pandas as pd

# Load the Netflix dataset from the CUHK-STAT3009 GitHub repository
# Repository link: https://github.com/statmlben/CUHK-STAT3009/tree/main/dataset/netflix

train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')

# Convert DataFrame to NumPy arrays
```

**Lasso Regression**

Given a dataset of feature-vectors $\mathbf{x}_i$ and corresponding ground truth scores $y_i$, Lasso regression seeks a sparse solution by minimizing the following objective function:

$$\text{argmin}_{\mathbf{\beta}} \ \frac{1}{n} \sum_{i=1}^n ( y_i - \mathbf{\beta}^T \mathbf{x}_i )^2 + \lambda \| \mathbf{\beta} \|_1, \quad \text{where } \| \mathbf{\beta} \|_1 = \sum_{j=1}^p |\beta_j|$$

This can be efficiently solved using `sklearn.linear_model.Lasso` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)).

### **Task: Lasso Matrix Factorization (Lasso_SVD)**

**Objective**

Implement a Lasso_SVD recommender system by solving the following optimization problem:

$$\boxed{(\widehat{\mathbf P}, \widehat{\mathbf Q}) = \text{argmin}_{\mathbf{P}, \mathbf{Q} } \frac{1}{|\Omega|} \sum_{(u,i) \in \Omega} ( r_{ui} - \mathbf{p}^\intercal_u \mathbf{q}_i  )^2 + \lambda \big(  \sum_{u=1}^n \|\mathbf{p}_u\|_1 + \sum_{i=1}^m \|\mathbf{q}_i\|_1 \big)}$$

**Implementation**

Create a class `Lasso_SVD` with two methods:

1. `Lasso_SVD.fit`: Fit the parameters $\mathbf{P}$ and $\mathbf{Q}$ by solving the optimization problem above using Lasso regression.
2. `Lasso_SVD.predict`: Predict ratings using the fitted parameters: $\widehat{r}_{ui} = \widehat{\mathbf{p}}^T_u \widehat{\mathbf{q}}_i$

**Hint**: Use Alternative Least Square (ALS) logic, where each subproblem is a Lasso regression that can be solved using `sklearn.linear_model.Lasso` (previously, we use `sklearn.linear_model.Ridge`).

**Evaluation**

Print the Root Mean Squared Error (RMSE) for the testing data using the following hyperparameters:

* $(\lambda = 0.1, K = 3)$
* $(\lambda = 0.3, K = 5)$

> Implement the `Lasso_SVD` class with the required methods. The correctness of the implementation will be evaluated based on the code structure and logic, not on the final evaluation results.

In [ ]:
## Your solution here

import numpy as np
import pandas as pd

# Load the Netflix dataset from the CUHK-STAT3009 GitHub repository
# Repository link: https://github.com/statmlben/CUHK-STAT3009/tree/main/dataset/netflix

train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')

## RS data casting with ML format
X_train = train[['user_id', 'movie_id']].values
y_train = train['rating'].values

X_test = test[['user_id', 'movie_id']].values
y_test = test['rating'].values


n_users = len(set(X_train[:, 0]).union(X_test[:, 0]))
n_items = len(set(X_train[:, 1]).union(X_test[:, 1]))

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import warnings
warnings.filterwarnings("ignore") # ignore all warnings from Lasso

# Define a Lasso_SVD class
class Lasso_SVD(BaseEstimator):
    def __init__(self, n_user, n_item, K, lam, iter_num, tol, lasso_iter_num=1000, verbose_interval=10):
        self.n_user = n_user
        self.n_item = n_item
        self.K = K
        self.lam = lam
        self.iter_num = iter_num
        self.tol = tol
        self.lasso_iter_num = lasso_iter_num
        self.verbose_interval = verbose_interval

        self.P = np.random.randn(n_user, K)
        self.Q = np.random.randn(n_item, K)

    def fit(self, X, y):
        n_obs = len(X)
        diff = 1.0

        obj_old = self.obj(X, y)

        user_indices = [np.where(X[:, 1] == item_id)[0] for item_id in range(self.n_item)]
        item_indices = [np.where(X[:, 0] == user_id)[0] for user_id in range(self.n_user)]

        for it in range(1, self.iter_num+1):
            # fix P, update Q
            for item_id in range(self.n_item):
                user_indices_tmp = user_indices[item_id]
                if len(user_indices_tmp) == 0:
                    self.Q[item_id] = np.zeros(self.K)
                else:
                    this_y = y[user_indices_tmp]
                    this_X = self.P[X[user_indices_tmp, 0]]
                    lasso = Lasso(alpha=self.lam, fit_intercept=False, max_iter=self.lasso_iter_num)
                    lasso.fit(this_X, this_y)
                    self.Q[item_id] = lasso.coef_

            # fix Q, update P
            for user_id in range(self.n_user):
                item_indices_tmp = item_indices[user_id]
                if len(item_indices_tmp) == 0:
                    self.P[user_id] = np.zeros(self.K)
                else:
                    this_y = y[item_indices_tmp]
                    this_X = self.Q[X[item_indices_tmp, 1]]
                    lasso = Lasso(alpha=self.lam, fit_intercept=False, max_iter=self.lasso_iter_num)
                    lasso.fit(this_X, this_y)
                    self.P[user_id] = lasso.coef_

            obj_new = self.obj(X, y)
            diff = np.abs(obj_old - obj_new)
            obj_old = obj_new
            if it % self.verbose_interval == 0:
                print(f"Iteration {it}, Objective: {obj_new}, Diff: {diff}")
            if diff < self.tol:
                print(f"difference in objective ({diff}) is smaller than tolerance({self.tol}), early stop")
                break
        return self

    def predict(self, X):
        y_pred = []
        for user_id, item_id in X:
            pred = np.dot(self.P[user_id], self.Q[item_id])
            y_pred.append(pred)
        return np.array(y_pred)

    def obj(self, X, y):
        y_pred = self.predict(X)
        mse = mean_squared_error(y,y_pred)
        penalty = np.sum(np.abs(self.P)) + np.sum(np.abs(self.Q))
        return mse + self.lam * penalty


In [ ]:
lasso_svd = Lasso_SVD(
    n_user=n_users,
    n_item=n_items,
    K=3,
    lam=0.1,
    iter_num=100,
    tol=1e-4
)

lasso_svd.fit(X_train, y_train)
y_pred = lasso_svd.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"lam=0.1 and K=3: Test RMSE = {rmse}")

Iteration 10, Objective: 1244.7967358808316, Diff: 27.402461173350275
Iteration 20, Objective: 1056.1405115682342, Diff: 15.042305485492761
Iteration 30, Objective: 995.7032181361453, Diff: 2.5142992043544155
Iteration 40, Objective: 982.9185824528912, Diff: 0.25800331515927155
Iteration 50, Objective: 981.6080986859253, Diff: 0.1107528143181753
Iteration 60, Objective: 982.1954023004232, Diff: 0.055029504326853385
Iteration 70, Objective: 981.6303633835165, Diff: 0.0025239514536679053
Iteration 80, Objective: 983.1163474664179, Diff: 0.1626884878154442
Iteration 90, Objective: 984.6589198687617, Diff: 0.1477314344158458
Iteration 100, Objective: 985.8551621372767, Diff: 0.10184386222204012
lam=0.1 and K=3: Test RMSE = 1.090166896814016


In [ ]:
lasso_svd = Lasso_SVD(
    n_user=n_users,
    n_item=n_items,
    K=5,
    lam=0.3,
    iter_num=100,
    tol=1e-4
)

lasso_svd.fit(X_train, y_train)
y_pred = lasso_svd.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"lam=0.3 and K=5: Test RMSE = {rmse}")

Iteration 10, Objective: 2597.091975850697, Diff: 1.2084675589962899
Iteration 20, Objective: 2615.2580717563774, Diff: 1.802120372837635
Iteration 30, Objective: 2627.90968580538, Diff: 0.8806042714977593
Iteration 40, Objective: 2633.648949015607, Diff: 0.37984908880889634
Iteration 50, Objective: 2635.940102967848, Diff: 0.03265076846400916
Iteration 60, Objective: 2636.524399345675, Diff: 0.06193541396442015
Iteration 70, Objective: 2636.9106992784045, Diff: 0.024680368204826664
Iteration 80, Objective: 2637.0649062503708, Diff: 0.009856344245690707
Iteration 90, Objective: 2637.126479099704, Diff: 0.003934855806164705
Iteration 100, Objective: 2637.1510596318094, Diff: 0.00157089687354528
lam=0.3 and K=5: Test RMSE = 1.0903198088661794


## **Q3: Kaggle Submission by Neural Networks**

**Task**

- Create an arbitrary Neural Network with Dense layers and Make a Submission to the Kaggle Competition: [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview)

- Paste the submission results screenshot into this Jupyter Notebook.

In [ ]:
## Your submission result here

## **Q4 (Bonus): Parallel Alternating Least Squares (ALS) for Matrix Factorization**


**Background**

Recall the item and user updates for `SVD` based on ALS:

$$\mathbf{q}^{(l+1)}_i = \left(\sum_{u \in \mathcal{U}_i} \mathbf{p}^{(l)}_u (\mathbf{p}^{(l)}_u)^T + \lambda |\Omega| \mathbf{I}\right)^{-1} \sum_{u \in \mathcal{U}_i} r_{ui} \mathbf{p}^{(l)}_u$$

$$\mathbf{p}^{(l+1)}_u = \left(\sum_{i \in \mathcal{I}_u} \mathbf{q}^{(l+1)}_i (\mathbf{q}^{(l+1)}_i)^\intercal + \lambda |\Omega| \mathbf{I}\right)^{-1} \sum_{i \in \mathcal{I}_u} r_{ui} \mathbf{q}^{(l+1)}_i$$

The key observation is that the updates for user-$u$ and item-$i$ are independent of other users and items, respectively. Therefore, they can be performed in parallel.

Suppose you have 100 users to update, the basic ALS updates user 1, user 2, ..., user 100 sequentially in a loop. Now, suppose you have 100 CPUs, the parallel ALS can update 100 users simultaneously by distributing each user to a different CPU, significantly reducing the computation time.

**Tasks**

1. **Parallelize the `SVD.fit` method**: Revise the `SVD.fit` method (available in [repo](https://github.com/statmlben/CUHK-STAT3009/blob/main/src/TabRS.py)) to allow parallel updating of $\mathbf{p}_u$ and $\mathbf{q}_i$ using Python libraries such as [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) or [pymp](https://github.com/classner/pymp).
2. **Compare computation times**: Compare the computation time for `SVD.fit` with and without parallel computing using the `%%time` magic command (see [ref](https://stackoverflow.com/questions/32565829/simple-way-to-measure-cell-execution-time-in-ipython-notebook)).

In [ ]:
# check number of CPUs in your PC/Node
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

In [ ]:
## You solution here

import time
from psvd import Sequential_SVD, Parallel_SVD

K = 5 # 50 or 100
lam = 0.001
iter_num = 20 # iter_num=100
tol = 1e-4 # tol=-1
max_cpus = 2 # cpus=8

sequential_svd = Sequential_SVD(
    n_users=n_users,
    n_items=n_items,
    K=K,
    lam=lam,
    iter_num=iter_num,
    tol=tol,
)

start = time.time()
sequential_svd.fit(X_train, y_train)
y_pred = sequential_svd.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Sequential SVD: RMSE={rmse:.4f}, Time={time.time() - start:.4f}s")

Iteration 1, Objective: 14.873000430823518, Diff: 31.78124931286706
Iteration 2, Objective: 14.873000430823518, Diff: 0.0
difference in objective (0.0) is smaller than tolerance(0.0001), early stop
Sequential SVD: RMSE=3.7729, Time=1.2118s


In [ ]:
parallel_svd = Parallel_SVD(
    n_users=n_users,
    n_items=n_items,
    K=K,
    lam=lam,
    iter_num=iter_num,
    tol=tol,
    max_cpus=max_cpus
)

start = time.time()
parallel_svd.fit(X_train, y_train)
y_pred = parallel_svd.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Parallel SVD: RMSE={rmse:.4f}, Time={time.time() - start:.4f}s")

Iteration 1, Objective: 15.000109709711758, Diff: 32.456375244420705
Iteration 2, Objective: 15.000109709711758, Diff: 0.0
difference in objective (0.0) is smaller than tolerance(0.0001), early stop
Parallel SVD: RMSE=3.7866, Time=4.9759s
